# 数据标注模块（Data Annotation Module）

数据标注模块基于Label Studio构建，为机器学习项目提供高效、灵活的数据标注解决方案。支持多种标注场景，特别适用于奖励模型和对话系统的数据准备工作。

## 🌟 核心特性

- **🎯 Label Studio深度集成**: 采用Label Studio作为标注界面，提供直观易用的标注体验
- **🔄 多场景数据支持**: 全面支持对话标注、质量评分、偏好排序等多种标注任务
- **🚀 快速部署启动**: 提供Docker和pip两种部署方式，支持一键启动标注服务
- **📊 模板化标注配置**: 内置RewardBench等常用标注模板，开箱即用
- **🔗 生态无缝集成**: 与RM Gallery数据处理管道深度集成，数据流转顺畅
- **📈 企业级批量处理**: 支持大规模数据的批量标注、导出和管理

## 🎯 应用场景

本模块主要适用于以下机器学习数据准备场景：

1. **奖励模型训练数据标注** - 为奖励模型训练准备高质量的偏好数据
2. **对话系统质量评估** - 评估和改进对话模型的输出质量
3. **偏好学习数据准备** - 构建用于偏好学习的比较数据集
4. **文本分类和情感分析** - 为监督学习任务准备标注数据

## 🚀 快速开始

### 1. 环境准备

确保系统已安装以下依赖：
```
label_studio==1.17.0
```

### 2. 启动Label Studio标注服务

使用以下命令启动标注服务：

```bash
# 使用Docker方式启动（推荐）
python ./rm_gallery/core/data/annotation/server.py start

# 使用pip方式启动
python ./rm_gallery/core/data/annotation/server.py start --use-pip

# 检查服务运行状态
python ./rm_gallery/core/data/annotation/server.py status

# 停止标注服务
python ./rm_gallery/core/data/annotation/server.py stop
```

服务启动成功后，控制台将显示以下信息：
```
============================================================
🚀 Label Studio Successfully Started!
============================================================
🌐 Web Interface: http://localhost:8080
📧 Username: admin@rmgallery.com
🔐 Password: RM-Gallery
📁 Data Directory: ./log/label_studio_logs
🐳 Deployment: Pip
============================================================
```

### 3. 验证服务状态

运行状态检查命令，确认服务正常运行：
```
==================================================
📊 Label Studio Status
==================================================
🌐 Server URL: http://localhost:8080
🚀 Deployment: PIP
🔌 Port: 8080
✅ Running
📁 Data Dir: ./log/label_studio_logs
👤 Username: admin@rmgallery.com
🔄 Process PIDs: 65727
🔌 Port PIDs: 65727
==================================================
```

### 4. 获取API Token

完成服务启动后，按以下步骤获取API Token：

1. 在浏览器中访问 http://localhost:8080
2. 使用以下凭据登录：
   - 用户名：`admin@rmgallery.com`
   - 密码：`RM-Gallery`
3. 点击左侧导航栏的"Organization"，进入API Tokens Settings
4. 将Personal Access Tokens和Legacy Tokens都设置为True
5. 点击右上角用户头像，选择"Account & Settings"
6. 在Access Token部分复制token值，这就是后续需要使用的API Token

## 💡 完整使用示例

下面通过一个完整的示例演示如何使用数据标注模块，包括数据导入、项目创建、标注执行和结果导出的全流程。

In [ ]:
"""
步骤1：导入数据并创建标注项目
"""

from rm_gallery.core.data.annotation.annotation import create_annotation_module
from rm_gallery.core.data.load.base import DataLoad
import rm_gallery.core.data     # 核心策略注册
import rm_gallery.gallery.data  # 扩展策略注册

# 替换为你从Label Studio获取的实际API Token
API_TOKEN = "1937f46f7b61906fac852236fe59de109abc0ed8"

# 步骤1.1：配置数据加载参数
load_config = {
    "path": "/Users/xielipeng/RM-Gallery/data/reward-bench-2/data/test-00000-of-00001.parquet",  # 替换为实际数据路径
    "limit": 1000,  # 限制加载的数据条数，避免首次测试加载过多数据
}

# 步骤1.2：创建数据加载器
loader = DataLoad(
    name="rewardbench2",           # 数据集标识名称
    load_strategy_type="local",    # 使用本地文件加载策略
    data_source="rewardbench2",    # 指定数据源格式转换器
    config=load_config             # 传入加载配置参数
)

# 步骤1.3：执行数据加载
print("正在加载数据...")
dataset = loader.run()
print(f"数据加载完成，共 {len(dataset.datas)} 条记录")

# 步骤1.4：配置标注项目参数
annotation_config = {
    "server_url": "http://localhost:8080",           # Label Studio服务地址
    "api_token": API_TOKEN,                          # API访问令牌
    "project_title": "RM Gallery Quality Annotation", # 项目显示名称
    "template_name": "rewardbench2",                 # 使用内置的RewardBench2模板
    "project_description": "基于RewardBench2模板的数据质量标注项目"
}

# 步骤1.5：创建标注模块实例
annotation_module = create_annotation_module(
    name="rm_gallery_annotation",
    **annotation_config
)

# 步骤1.6：创建标注项目并导入数据
print("正在创建标注项目...")
result = annotation_module.run(dataset, create_new_project=True)

if result:
    project_url = f"{result.metadata['annotation_server_url']}/projects/{result.metadata['annotation_project_id']}"
    print(f"✅ 标注项目创建成功！")
    print(f"🌐 项目访问地址: {project_url}")
    print(f"📊 项目ID: {result.metadata['annotation_project_id']}")
else:
    print("❌ 创建标注项目失败，请检查配置和网络连接")

正在加载数据...
数据加载完成，共 1000 条记录
正在创建标注项目...
✅ 标注项目创建成功！
🌐 项目访问地址: http://localhost:8080/projects/9
📊 项目ID: 9


In [ ]:
"""
步骤2：进行数据标注

项目创建成功后，你可以：
1. 访问上面输出的项目地址
2. 使用 admin@rmgallery.com / RM-Gallery 登录
3. 在标注界面中对数据进行标注
4. 标注完成后，执行下一步导出标注结果

注意：在实际使用中，你需要手动完成标注工作，然后再运行下面的导出代码。
"""

# 此处为标注操作的占位符
# 实际标注工作需要在Label Studio的Web界面中完成
print("📝 请在Label Studio界面中完成数据标注工作")
print("💡 标注完成后，运行下面的代码导出结果")

In [ ]:
"""
步骤3：导出标注结果
"""

from rm_gallery.core.data.annotation.annotation import create_annotation_module
from rm_gallery.core.data.export import create_export_module
import rm_gallery.core.data     # 核心策略注册
import rm_gallery.gallery.data  # 扩展策略注册

# 使用与创建项目时相同的API Token
API_TOKEN = "1937f46f7b61906fac852236fe59de109abc0ed8"

# 步骤3.1：重新创建标注模块实例
annotation_module = create_annotation_module(
    name="rm_gallery_annotation",
    template_name="rewardbench2",
    api_token=API_TOKEN
)

# 步骤3.2：设置项目ID（从步骤1的输出中获取）
annotation_module.project_id = 3  # 替换为实际的project_id

# 步骤3.3：从Label Studio导出标注数据
print("正在导出标注数据...")
try:
    annotated_dataset = annotation_module.export_annotations_to_dataset()
    print(f"✅ 标注数据导出成功，共 {len(annotated_dataset.datas)} 条已标注记录")
except Exception as e:
    print(f"❌ 导出失败: {e}")
    annotated_dataset = None

# 步骤3.4：配置文件导出器
if annotated_dataset:
    exporter = create_export_module(
        name="annotation_exporter",
        config={
            "output_dir": "./exports",          # 导出文件存放目录
            "formats": ["jsonl"]        # 支持多种导出格式
        }
    )
    
    # 步骤3.5：执行数据导出
    print("正在保存标注结果到文件...")
    export_result = exporter.run(annotated_dataset)
    
    if export_result:
        print("✅ 标注结果已保存到 ./exports 目录")
        print(f"📊 数据集信息: {annotated_dataset.name}")
        print(f"📁 包含 {len(annotated_dataset.datas)} 条标注数据")
    else:
        print("❌ 文件导出失败")
    
    # 显示部分数据预览
    if annotated_dataset.datas:
        print("\n📋 数据预览:")
        sample = annotated_dataset.datas[0]
        print(f"  - 样本ID: {sample.unique_id}")
        print(f"  - 标注状态: {sample.metadata.get('annotation_status', 'unknown')}")
        print(f"  - 输出数量: {len(sample.output) if sample.output else 0}")


正在导出标注数据...
✅ 标注数据导出成功，共 1 条已标注记录
正在保存标注结果到文件...
✅ 标注结果已保存到 ./exports 目录
📊 数据集信息: rm_gallery_annotation_annotations
📁 包含 1 条标注数据

📋 数据预览:
  - 样本ID: 901
  - 标注状态: completed
  - 输出数量: 3



## 📋 内置标注模板

系统提供以下预配置的标注模板，开箱即用，见`rm_gallery/gallery/data/annotation/`：

| 模板名称 | 模板标识 | 来源 | 说明 |
|---------|---------|---------|------|
| RewardBenchAnnotationTemplate | `rewardbench` | RewardBench | 支持2选一的质量评分和排序 |
| RewardBench2AnnotationTemplate | `rewardbench2` | RewardBench2 | 支持4选一的质量评分和排序 |

## 🛠️ 自定义标注模板开发

如果内置模板不满足你的标注需求，可以按以下步骤开发自定义模板：

### 步骤1：创建模板类

在 `rm_gallery/gallery/data/annotation/` 目录下创建新的模板文件，继承`BaseAnnotationTemplate`基类：

```python
from rm_gallery.core.data.annotation.template import BaseAnnotationTemplate, AnnotationTemplateRegistry

@AnnotationTemplateRegistry.register("custom_template")
class CustomAnnotationTemplate(BaseAnnotationTemplate):
    @property
    def label_config(self) -> str:
        """
        定义Label Studio标注界面配置
        使用Label Studio的XML配置语法
        """
        return """
        <View>
            <Text name="question" value="$question"/>
            <Choices name="quality" toName="question" choice="single-radio">
                <Choice value="excellent" background="green"/>
                <Choice value="good" background="blue"/>
                <Choice value="fair" background="yellow"/>
                <Choice value="poor" background="red"/>
            </Choices>
            <Rating name="score" toName="question" maxRating="10" />
            <TextArea name="comments" toName="question" 
                     placeholder="请输入评价理由..." rows="3"/>
        </View>
        """
    
    def process_annotations(self, annotation_data):
        """
        处理从Label Studio获取的标注数据
        将原始标注数据转换为结构化格式
        """
        processed_data = {
            "quality_rating": annotation_data.get("choices", {}).get("quality", {}).get("choices", []),
            "numerical_score": annotation_data.get("rating", {}).get("score", {}).get("rating", 0),
            "textual_feedback": annotation_data.get("textarea", {}).get("comments", {}).get("text", [""])[0]
        }
        return processed_data
```

### 步骤2：注册模板

在 `rm_gallery/gallery/data/__init__.py` 文件中导入你的模板类以完成注册：

```python
# 导入自定义标注模板
from rm_gallery.gallery.data.annotation.custom_template import CustomAnnotationTemplate
```

### 步骤3：使用自定义模板

在创建标注模块时，指定你的自定义模板名称：

```python
annotation_module = create_annotation_module(
    name="custom_annotation_project",
    template_name="custom_template",  # 使用你注册的模板名称
    # ... 其他配置参数
)
```

## 📊 数据格式规范

### 输入数据要求

- **DataSample标准格式**  

### 输出数据格式

- **标注结果集成**: 标注数据自动添加到DataSample的`label`字段中
- **原始数据保护**: 保持输入数据的完整性和原始结构
- **丰富元数据**: 包含标注项目ID、标注状态、时间戳等追踪信息
- **多格式导出**: 支持JSON、JSONL等多种导出格式

### 数据结构示例

```python
# 标注后的DataSample结构示例
data_sample = DataSample(
    unique_id="sample_001",
    input=[...],  # 原始输入数据
    output=[...], # 原始输出数据（如果有）
    metadata={
        "annotation_status": "completed",
        "annotation_project_id": 123,
        "annotator_id": "user@example.com"
    }
)

# 每个output中的label字段包含标注结果
output.label = {
    "annotation_data": {
        "ratings": {...},      # 评分数据
        "choices": {...},      # 选择数据
        "text_areas": {...}    # 文本输入数据
    },
    "processed": {...}         # 经过模板处理的结构化数据
}
```

## 🔧 故障排除指南

### 常见问题及解决方案

#### 1. Label Studio服务启动失败

**问题表现**: 服务无法启动或启动后无法访问

**解决步骤**:
```bash
# 检查端口是否被占用
lsof -i :8080

# 如果端口被占用，使用不同端口启动
python ./rm_gallery/core/data/annotation/server.py start --port 8081

# 查看详细启动日志
python ./rm_gallery/core/data/annotation/server.py start --data-dir ./custom_log --verbose

# 清理之前的数据目录（谨慎使用）
rm -rf ./log/label_studio_logs
```

#### 2. API Token获取失败

**问题表现**: 无法在界面中找到API Token

**解决步骤**:
1. 确保已正确登录Label Studio界面
2. 检查用户权限设置，确保具有管理员权限
3. 在Organization设置中启用API Tokens功能
4. 如果仍无法获取，尝试重新创建用户账户

#### 3. 数据导入失败

**问题表现**: 项目创建成功但数据无法导入

**解决步骤**:
- 检查数据格式是否符合要求
- 验证API Token是否正确
- 确认网络连接正常
- 检查数据大小是否超出限制

#### 4. 标注结果导出异常

**问题表现**: 导出的数据不完整或格式异常

**解决步骤**:
- 确认所有数据都已完成标注
- 检查导出路径的写入权限
- 验证模板的process_annotations方法是否正确实现

### 获取技术支持

如果遇到无法解决的问题，可以通过以下方式获取帮助：

1. **查看日志文件**: `./log/label_studio_logs/` 目录下的日志文件
2. **检查系统状态**: 运行 `python ./rm_gallery/core/data/annotation/server.py status`
3. **重启服务**: 完全停止服务后重新启动
4. **社区支持**: 在项目仓库中提交Issue，提供详细的错误信息和复现步骤